# Install

In [ ]:
!pip3 install hydra-core

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install datasets

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install sentence_transformers

In [7]:
# !pip install git+https://github.com/huggingface/transformers
# !pip install sentencepiece
# !pip install bitsandbytes
# !pip install datasets
# !pip install loralib

In [ ]:
# !pip3 install accelerate

In [9]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# HuggingChat
!pip3 install hugchat

In [ ]:
# # DeepL
# !pip3 install deepl

In [ ]:
# Google Translate
!pip install googletrans==3.1.0a0

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Sbert

In [ ]:
import os
os.chdir('drive/MyDrive/MemeProject/src/sbert')

In [ ]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

data load

In [ ]:
emb_path = "/content/drive/MyDrive/MemeProject/data/sbert/"

In [ ]:
# reference
def data_load(path):
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line.rstrip("\n|\r")))
    data = pd.DataFrame(data)
    return data

df_ref = data_load(emb_path + "ref.json")

In [ ]:
# ref load
emb_list = os.listdir(emb_path)
emb_list = sorted([i for i in emb_list if ".npy" in i])
print(emb_list)

print(f"file load start")
ref_embs = [np.load(emb_path + i).tolist() for i in tqdm(emb_list)]
print(f"file load end")
ref_emb = torch.Tensor(sum(ref_embs, []))
print(f"tensor end")
print(ref_emb.shape)

['ref_emb_1.npy', 'ref_emb_2.npy', 'ref_emb_3.npy', 'ref_emb_4.npy', 'ref_emb_5.npy', 'ref_emb_6.npy']
file load start


100%|██████████| 6/6 [01:02<00:00, 10.38s/it]


file load end
tensor end
torch.Size([528406, 768])


model load

In [ ]:
model_sbert = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")

input

In [ ]:
input = '커피 한잔 하고 싶다~'

In [ ]:
query_emb = model_sbert.encode([input], convert_to_tensor=True)
hits = util.semantic_search(query_emb, ref_emb, top_k=1)

# output
print(f"hits :{hits}")
print(f"score : {hits[0][0]['score']}")
print(df_ref[df_ref.u == df_ref.iloc[hits[0][0]["corpus_id"]]["u"]])
df_result = (
    df_ref[df_ref.u == df_ref.iloc[hits[0][0]["corpus_id"]]["u"]]
    .sample(frac=1)
    .reset_index(drop=True)
)
print(f"밈 대답 : {df_result.iloc[0]['meme']}")

hits :[[{'corpus_id': 97860, 'score': 0.8415704965591431}]]
score : 0.8415704965591431
         num             u                      meme
97860  97860  나도 커피 먹고 싶음!              카페인 수혈 가보자고!
97861  97861  나도 커피 먹고 싶음!              카페인 수혈 가즈아~~
97862  97862  나도 커피 먹고 싶음!  카페인 수혈 하러 갈 파티원 모집 (1/N)
97863  97863  나도 커피 먹고 싶음!           카페인 수혈 당장 진행시켜!
97864  97864  나도 커피 먹고 싶음!          카페인 수혈 가야G 가야G~!
97865  97865  나도 커피 먹고 싶음!          카페인 수혈 같이 하러 가실?
97866  97866  나도 커피 먹고 싶음!               커피 수혈 가보자고!
97867  97867  나도 커피 먹고 싶음!               커피 수혈 가즈아~~
97868  97868  나도 커피 먹고 싶음!   커피 수혈 하러 갈 파티원 모집 (1/N)
97869  97869  나도 커피 먹고 싶음!            커피 수혈 당장 진행시켜!
97870  97870  나도 커피 먹고 싶음!           커피 수혈 가야G 가야G~!
97871  97871  나도 커피 먹고 싶음!           커피 수혈 같이 하러 가실?
밈 대답 : 카페인 수혈 가야G 가야G~!


In [ ]:
# # Tokenize sentences
# sent_encoded = tokenizer(
#     input,
#     padding=True,
#     max_length=512,
#     truncation=True,
#     return_tensors="pt",
# )

# # Compute token embeddings
# with torch.no_grad():
#     output = model(**sent_encoded)

# # Perform pooling. In this case, mean pooling.
# input_emb = mean_pooling(output, sent_encoded["attention_mask"])

# # cosine similarity
# cosine_similarities = linear_kernel(input_emb, ref_emb).flatten()
# related_docs_indices = cosine_similarities.argsort()[:-5:-1]

# print(f"cosine similarities : {cosine_similarities}")
# print(f"코사인 유사도 연산 결과 : {related_docs_indices}")
# print(f"입력 문장 : {input[0]}")
# print(f"유사 문장 : {data.iloc[related_docs_indices[0]]['u']}")
# print(f"밈 답변 : {data.iloc[related_docs_indices[0]]['meme']}")

# KoAlpaca-Polyglot-12.8B

##### Pipeline

In [ ]:
import torch
from transformers import AutoModelForCausalLM, pipeline

In [ ]:
model_name = 'beomi/KoAlpaca-Polyglot-12.8B'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=5120, out_features=15360, bias=True)
          (dense): Linear(in_features=5120, out_features=5120, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear(in_features=20480, out_features=5120, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=5120, out_features=30003, bias=False)
)

In [ ]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=model_name,
    device=0
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
def ask(x, context='', is_input_full=False):
    ans = pipe(
        f"### 질문: {x}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {x}\n\n### 답변:",
        do_sample=True,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.5,
        return_full_text=False,
        eos_token_id=2,
    )
    print(ans[0]['generated_text'])

Inference

In [ ]:
context = """너는 나랑 일상적인 대화를 할거야.
내가 하는 말에 반말로 대답해줘.
"""
prompt = """너 이름이 뭐야?"""

In [ ]:
# prompt = """너 이름이 뭐야? 일상적인 대화로 반말로 대답해줘."""

In [ ]:
ask(prompt, context)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


너는 나랑 일상적인 대화를 할 거야. 내가 하는 말에 반말로 대답해줘.


##### AutoModel

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = 'beomi/KoAlpaca-Polyglot-12.8B'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attention): GPTNeoXAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=5120, out_features=15360, bias=True)
          (dense): Linear(in_features=5120, out_features=5120, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear(in_features=20480, out_features=5120, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=5120, out_features=30003, bias=False)
)

In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [ ]:
def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.7):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)

    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

    return gen_text.replace(x, '')

Chat

In [ ]:
user_input = "오늘 날씨가 뭐야?"
prompt = f"너의 이름은 대춘이고 너의 성격은 반말로 까칠하게 대답하는 것이야. 대답을 오직 존댓말이 아닌 반말로만 해줘."

In [ ]:
generated_text = gen(prompt, user_input)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
print(generated_text.split("### 명령어:")[0].replace('\n', ''))

오늘 날씨는 맑고 더워.입력된 값을 출력하는 것이 명령어입니다. 오늘 날씨는 맑고 더워!


# HuggingChat

In [13]:
import os
os.chdir('drive/MyDrive/MemeProject/src/sbert')

In [14]:
import config as cfg
from hugchat import hugchat
from hugchat.login import Login

입력 유저 발화

In [15]:
user_input = "내 이름은 박성환이야!"

##### DeepL

In [ ]:
import deepl

In [ ]:
translator = deepl.Translator(cfg.auth_key)

In [ ]:
input = translator.translate_text(user_input, target_lang="EN-US")

In [ ]:
print(input.text)

##### Google Translate

In [16]:
from googletrans import Translator

In [17]:
google = Translator()

In [23]:
trans = google.translate(user_input, dest="en")

In [24]:
trans.text

'My name is Park Seong-hwan!'

##### HuggingChat

In [25]:
# Log in to huggingface and grant authorization to huggingchat
sign = Login(cfg.email, cfg.passwd)
cookies = sign.login()

In [26]:
# # Save cookies to usercookies/<email>.json
# sign.saveCookies()

In [27]:
# Create a ChatBot
chatbot = hugchat.ChatBot(cookies=cookies.get_dict())  # or cookie_path="usercookies/<email>.json"
print(chatbot.chat(trans.text))

Hello Mr. Park Seong-hwan I am here to assist you with any questions you may have, How can I help?


In [28]:
result = google.translate(chatbot.chat(trans.text), dest="ko")  # 초록 번역

In [29]:
print(result.text)

만나서 반가워요, 미스터 박. 오늘 도와드릴 일이 있습니까?


In [ ]:
# # Create a new conversation
# id = chatbot.new_conversation()
# chatbot.change_conversation(id)

# # Get conversation list
# conversation_list = chatbot.get_conversation_list()